In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import zscore
import plotly.express as px

# Load the Excel data
df = pd.read_excel("rrg_sectoral.xlsx", parse_dates=["DATES"], index_col="DATES")

# Define sectors and benchmark
sectors = ["FMCG"]
#sectors = ["Auto", "Bank", "Chemicals", "Consumer Durables", "FMCG", "Healthcare", "Pharma", "IT", "Metal"]
benchmark = "NIFTY Index"
slice_window = 5
# Calculate Relative Strength Ratio (price of sector / price of benchmark)
rs_ratios = df[sectors].div(df[benchmark], axis=0)
short_period = 5
rs_ratios =rs_ratios.iloc[::slice_window]
rs_momentum = (rs_ratios / rs_ratios.shift(short_period) - 1) * 100
rs_momentum = rs_momentum.iloc[::slice_window]
rolling_window = 12

rs_ratios_normalized = rs_ratios.rolling(window=rolling_window).apply(
    lambda x: (x.iloc[-1] - x.mean()) / x.std() if len(x.dropna()) > 1 else 0
)
# Chop off the in-betweens: keep only points at each 7-day interval, starting from the latest date backwards
# rs_ratios_normalized = rs_ratios_normalized.iloc[::-1].iloc[::7].iloc[::-1]
# rs_ratios_normalized = rs_ratios_normalized.iloc[::slice_window]  # Reverse, slice every 7th, reverse back
print(rs_ratios_normalized)
rs_momentum_normalized = rs_momentum.rolling(window=rolling_window).apply(
    lambda x: (x.iloc[-1] - x.mean()) / x.std() if len(x.dropna()) > 1 else 0
)
# Chop off the in-betweens: keep only points at each 7-day interval, starting from the latest date backwards
# rs_momentum_normalized = rs_momentum_normalized.iloc[::-1].iloc[::7].iloc[::-1]
# rs_momentum_normalized = rs_momentum_normalized.iloc[::slice_window]
# Select the latest date for the plot
target_date = rs_momentum_normalized.index[-1]
rs_ratios_latest = rs_ratios_normalized.loc[target_date]
rs_momentum_latest = rs_momentum_normalized.loc[target_date]

# Create the RRG plot
fig = go.Figure()

# Define colors for sectors to ensure distinct visuals
colors = px.colors.qualitative.Plotly[:len(sectors)]

# Plot scatter points and tails for each sector
for sector, color in zip(sectors, colors):
    # Tail: Plot the last 21 days (approx. 1 month) for trend
    tail_length = 30
    tail_data_rs = rs_ratios_normalized[sector][-tail_length:].dropna()
    tail_data_momentum = rs_momentum_normalized[sector][-tail_length:].dropna()
    
    if len(tail_data_rs) > 1:
        # Plot the tail line
        fig.add_trace(
            go.Scatter(
                x=tail_data_rs,
                y=tail_data_momentum,
                mode="lines",
                line=dict(color=color, width=3, dash="dash"),
                showlegend=False,
                name=f"{sector}_tail"
            )
        )
        
        # Add arrows along the path to show direction
        # Calculate arrow positions (every few points along the tail)
        arrow_interval = max(1,0)  # Show ~5 arrows max
        
        for i in range(arrow_interval, len(tail_data_rs), arrow_interval):
            if i < len(tail_data_rs) - 1:
                # Current and next points
                x1, y1 = tail_data_rs.iloc[i-1], tail_data_momentum.iloc[i-1]
                x2, y2 = tail_data_rs.iloc[i], tail_data_momentum.iloc[i]
                
                # Calculate arrow direction
                dx = x2 - x1
                dy = y2 - y1
                
                # Normalize arrow length
                arrow_length = 0.1
                if dx != 0 or dy != 0:
                    length = np.sqrt(dx**2 + dy**2)
                    if length > 0:
                        dx_norm = (dx / length) * arrow_length
                        dy_norm = (dy / length) * arrow_length
                        
                        # Add arrow annotation
                        fig.add_annotation(
                            x=x2,
                            y=y2,
                            ax=x2 - dx_norm,
                            ay=y2 - dy_norm,
                            xref="x",
                            yref="y",
                            axref="x",
                            ayref="y",
                            showarrow=True,
                            arrowhead=2,
                            arrowsize=1,
                            arrowwidth=2,
                            arrowcolor=color,
                            bgcolor="rgba(0,0,0,0)",
                            bordercolor="rgba(0,0,0,0)"
                        )
    
    # Scatter point for the latest date (current position)
    fig.add_trace(
        go.Scatter(
            x=[rs_ratios_latest[sector]],
            y=[rs_momentum_latest[sector]],
            mode="markers+text",
            name=sector,
            marker=dict(size=15, color=color, symbol="circle", 
                       line=dict(width=2, color="white")),
            text=[sector],
            textposition="top center",
            textfont=dict(size=12, color="black")
        )
    )
    
    # Add a larger arrow at the very end to show most recent direction
    if len(tail_data_rs) > 1:
        # Last two points for final direction
        x1, y1 = tail_data_rs.iloc[-2], tail_data_momentum.iloc[-2]
        x2, y2 = tail_data_rs.iloc[-1], tail_data_momentum.iloc[-1]
        
        dx = x2 - x1
        dy = y2 - y1
        
        if dx != 0 or dy != 0:
            length = np.sqrt(dx**2 + dy**2)
            if length > 0:
                arrow_length = 0.15
                dx_norm = (dx / length) * arrow_length
                dy_norm = (dy / length) * arrow_length
                
                fig.add_annotation(
                    x=x2,
                    y=y2,
                    ax=x2 - dx_norm,
                    ay=y2 - dy_norm,
                    xref="x",
                    yref="y",
                    axref="x",
                    ayref="y",
                    showarrow=True,
                    arrowhead=3,
                    arrowsize=2,
                    arrowwidth=3,
                    arrowcolor=color,
                    bgcolor="rgba(0,0,0,0)",
                    bordercolor="rgba(0,0,0,0)"
                )

# Add quadrant lines and background colors
fig.add_shape(type="line", x0=0, y0=-3, x1=0, y1=3, line=dict(color="black", width=1))
fig.add_shape(type="line", x0=-3, y0=0, x1=3, y1=0, line=dict(color="black", width=1))

# Add colored quadrant backgrounds
fig.add_shape(type="rect", x0=0, y0=0, x1=3, y1=3, fillcolor="lightgreen", opacity=0.3, layer="below")
fig.add_shape(type="rect", x0=0, y0=0, x1=3, y1=-3, fillcolor="yellow", opacity=0.3, layer="below")
fig.add_shape(type="rect", x0=0, y0=0, x1=-3, y1=-3, fillcolor="lightcoral", opacity=0.3, layer="below")
fig.add_shape(type="rect", x0=0, y0=0, x1=-3, y1=3, fillcolor="lightblue", opacity=0.3, layer="below")

# Add quadrant labels
fig.add_annotation(x=1.5, y=1.5, text="Leading<br>(Strong & Improving)", 
                  showarrow=False, font=dict(size=10, color="darkgreen"))
fig.add_annotation(x=1.5, y=-1.5, text="Weakening<br>(Strong & Deteriorating)", 
                  showarrow=False, font=dict(size=10, color="orange"))
fig.add_annotation(x=-1.5, y=-1.5, text="Lagging<br>(Weak & Deteriorating)", 
                  showarrow=False, font=dict(size=10, color="darkred"))
fig.add_annotation(x=-1.5, y=1.5, text="Improving<br>(Weak & Improving)", 
                  showarrow=False, font=dict(size=10, color="blue"))

# Update layout
fig.update_layout(
    title=f"Relative Rotation Graph vs. {benchmark} ({target_date.strftime('%Y-%m-%d')})",
    xaxis_title="Relative Strength Ratio (Z-score)",
    yaxis_title="Relative Strength Momentum (Z-score)",
    showlegend=True,
    width=900,
    height=700,
    xaxis=dict(range=[-3, 3], zeroline=True),
    yaxis=dict(range=[-3, 3], zeroline=True)
)

# Show plot
fig.show()
#curvature and toggle , application


                FMCG
DATES               
2018-01-01       NaN
2018-01-08       NaN
2018-01-15       NaN
2018-01-22       NaN
2018-01-30       NaN
...              ...
2025-03-28 -1.076800
2025-04-07  1.316745
2025-04-16  1.221323
2025-04-24  0.246148
2025-05-02 -0.363085

[364 rows x 1 columns]


In [151]:
import plotly.graph_objects as go

# Parameter to control the number of past days to plot
past_days = 10  # Adjust this value as needed (e.g., 126 for 6 months, 63 for 3 months)

# Filter data for the specified past days
start_date = rs_ratios_normalized.index[-past_days]
rs_ratios_filtered = rs_ratios_normalized.loc[start_date:]
rs_momentum_filtered = rs_momentum_normalized.loc[start_date:]

# Create the plot with dual y-axes
fig = go.Figure()

# Add RS Ratio (left y-axis)
fig.add_trace(
    go.Scatter(
        x=rs_ratios_filtered.index,
        y=rs_ratios_filtered["FMCG"],
        mode="lines",
        name="RS Ratio",
        line=dict(color="blue")
    )
)

# Add RS Momentum (right y-axis)
fig.add_trace(
    go.Scatter(
        x=rs_momentum_filtered.index,
        y=rs_momentum_filtered["FMCG"],
        mode="lines",
        name="RS Momentum",
        line=dict(color="orange"),
        yaxis="y2"
    )
)

# Update layout with dual y-axes
fig.update_layout(
    title=f"Time Series of RS Ratio and RS Momentum for Pharma (Last {past_days} Days)",
    xaxis_title="Date",
    yaxis=dict(title="Relative Strength Ratio (Z-score)", tickfont=dict(color="blue")),
    yaxis2=dict(title="Relative Strength Momentum (Z-score)", tickfont=dict(color="orange"), overlaying="y", side="right"),
    width=1000,
    height=500,
    showlegend=True,
    hovermode="x unified"
)

# Show plot
fig.show()

In [154]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import zscore
import plotly.express as px

# Load the Excel data
df = pd.read_excel("rrg_data.xlsx", parse_dates=["DATES"], index_col="DATES")

# Define sectors and benchmark
sectors = [ "NSELV30 Index"]
# sectors = ["NSELV30 Index", "N200MOMP Index", "NSEQ30TR Index", "NSENV20 Index"]
benchmark = "NIFTY Index"

rs_ratios = df[sectors].div(df[benchmark], axis=0)
short_period = 7
rs_momentum = (rs_ratios / rs_ratios.shift(short_period) - 1) * 100
rolling_window = 84

rs_ratios_normalized = rs_ratios.rolling(window=rolling_window).apply(
    lambda x: (x.iloc[-1] - x.mean()) / x.std() if len(x.dropna()) > 1 else 0
)
# Chop off the in-betweens: keep only points at each 7-day interval, starting from the latest date backwards
# rs_ratios_normalized = rs_ratios_normalized.iloc[::-1].iloc[::7].iloc[::-1]
rs_ratios_normalized = rs_ratios_normalized.iloc[::7]  # Reverse, slice every 7th, reverse back
print(rs_ratios_normalized)
rs_momentum_normalized = rs_momentum.rolling(window=rolling_window).apply(
    lambda x: (x.iloc[-1] - x.mean()) / x.std() if len(x.dropna()) > 1 else 0
)
# Chop off the in-betweens: keep only points at each 7-day interval, starting from the latest date backwards
# rs_momentum_normalized = rs_momentum_normalized.iloc[::-1].iloc[::7].iloc[::-1]
rs_momentum_normalized = rs_momentum_normalized.iloc[::7]
# Select the latest date for the plot
target_date = rs_momentum_normalized.index[-1]
rs_ratios_latest = rs_ratios_normalized.loc[target_date]
rs_momentum_latest = rs_momentum_normalized.loc[target_date]

# Create the RRG plot
fig = go.Figure()

# Define colors for sectors to ensure distinct visuals
colors = px.colors.qualitative.Plotly[:len(sectors)]

# Plot scatter points and tails for each sector
for sector, color in zip(sectors, colors):
    # Tail: Plot the last 21 days (approx. 1 month) for trend
    tail_length = 30
    tail_data_rs = rs_ratios_normalized[sector][-tail_length:].dropna()
    tail_data_momentum = rs_momentum_normalized[sector][-tail_length:].dropna()
    
    if len(tail_data_rs) > 1:
        # Plot the tail line
        fig.add_trace(
            go.Scatter(
                x=tail_data_rs,
                y=tail_data_momentum,
                mode="lines",
                line=dict(color=color, width=3, dash="dash"),
                showlegend=False,
                name=f"{sector}_tail"
            )
        )
        
        # Add arrows along the path to show direction
        # Calculate arrow positions (every few points along the tail)
        arrow_interval = max(1,0)  # Show ~5 arrows max
        
        for i in range(arrow_interval, len(tail_data_rs), arrow_interval):
            if i < len(tail_data_rs) - 1:
                # Current and next points
                x1, y1 = tail_data_rs.iloc[i-1], tail_data_momentum.iloc[i-1]
                x2, y2 = tail_data_rs.iloc[i], tail_data_momentum.iloc[i]
                
                # Calculate arrow direction
                dx = x2 - x1
                dy = y2 - y1
                
                # Normalize arrow length
                arrow_length = 0.1
                if dx != 0 or dy != 0:
                    length = np.sqrt(dx**2 + dy**2)
                    if length > 0:
                        dx_norm = (dx / length) * arrow_length
                        dy_norm = (dy / length) * arrow_length
                        
                        # Add arrow annotation
                        fig.add_annotation(
                            x=x2,
                            y=y2,
                            ax=x2 - dx_norm,
                            ay=y2 - dy_norm,
                            xref="x",
                            yref="y",
                            axref="x",
                            ayref="y",
                            showarrow=True,
                            arrowhead=2,
                            arrowsize=1,
                            arrowwidth=2,
                            arrowcolor=color,
                            bgcolor="rgba(0,0,0,0)",
                            bordercolor="rgba(0,0,0,0)"
                        )
    
    # Scatter point for the latest date (current position)
    fig.add_trace(
        go.Scatter(
            x=[rs_ratios_latest[sector]],
            y=[rs_momentum_latest[sector]],
            mode="markers+text",
            name=sector,
            marker=dict(size=15, color=color, symbol="circle", 
                       line=dict(width=2, color="white")),
            text=[sector],
            textposition="top center",
            textfont=dict(size=12, color="black")
        )
    )
    
    # Add a larger arrow at the very end to show most recent direction
    if len(tail_data_rs) > 1:
        # Last two points for final direction
        x1, y1 = tail_data_rs.iloc[-2], tail_data_momentum.iloc[-2]
        x2, y2 = tail_data_rs.iloc[-1], tail_data_momentum.iloc[-1]
        
        dx = x2 - x1
        dy = y2 - y1
        
        if dx != 0 or dy != 0:
            length = np.sqrt(dx**2 + dy**2)
            if length > 0:
                arrow_length = 0.15
                dx_norm = (dx / length) * arrow_length
                dy_norm = (dy / length) * arrow_length
                
                fig.add_annotation(
                    x=x2,
                    y=y2,
                    ax=x2 - dx_norm,
                    ay=y2 - dy_norm,
                    xref="x",
                    yref="y",
                    axref="x",
                    ayref="y",
                    showarrow=True,
                    arrowhead=3,
                    arrowsize=2,
                    arrowwidth=3,
                    arrowcolor=color,
                    bgcolor="rgba(0,0,0,0)",
                    bordercolor="rgba(0,0,0,0)"
                )

# Add quadrant lines and background colors
fig.add_shape(type="line", x0=0, y0=-3, x1=0, y1=3, line=dict(color="black", width=1))
fig.add_shape(type="line", x0=-3, y0=0, x1=3, y1=0, line=dict(color="black", width=1))

# Add colored quadrant backgrounds
fig.add_shape(type="rect", x0=0, y0=0, x1=3, y1=3, fillcolor="lightgreen", opacity=0.3, layer="below")
fig.add_shape(type="rect", x0=0, y0=0, x1=3, y1=-3, fillcolor="yellow", opacity=0.3, layer="below")
fig.add_shape(type="rect", x0=0, y0=0, x1=-3, y1=-3, fillcolor="lightcoral", opacity=0.3, layer="below")
fig.add_shape(type="rect", x0=0, y0=0, x1=-3, y1=3, fillcolor="lightblue", opacity=0.3, layer="below")

# Add quadrant labels
fig.add_annotation(x=1.5, y=1.5, text="Leading<br>(Strong & Improving)", 
                  showarrow=False, font=dict(size=10, color="darkgreen"))
fig.add_annotation(x=1.5, y=-1.5, text="Weakening<br>(Strong & Deteriorating)", 
                  showarrow=False, font=dict(size=10, color="orange"))
fig.add_annotation(x=-1.5, y=-1.5, text="Lagging<br>(Weak & Deteriorating)", 
                  showarrow=False, font=dict(size=10, color="darkred"))
fig.add_annotation(x=-1.5, y=1.5, text="Improving<br>(Weak & Improving)", 
                  showarrow=False, font=dict(size=10, color="blue"))

# Update layout
fig.update_layout(
    title=f"Relative Rotation Graph vs. {benchmark} ({target_date.strftime('%Y-%m-%d')})",
    xaxis_title="Relative Strength Ratio (Z-score)",
    yaxis_title="Relative Strength Momentum (Z-score)",
    showlegend=True,
    width=900,
    height=700,
    xaxis=dict(range=[-3, 3], zeroline=True),
    yaxis=dict(range=[-3, 3], zeroline=True)
)

# Show plot
fig.show()
#curvature and toggle , application

            NSELV30 Index
DATES                    
2018-01-01            NaN
2018-01-08            NaN
2018-01-15            NaN
2018-01-22            NaN
2018-01-29            NaN
...                   ...
2025-04-07      -0.046052
2025-04-14      -0.120270
2025-04-21      -1.465469
2025-04-28      -0.972183
2025-05-05      -1.378415

[384 rows x 1 columns]


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Initialize Dash app with external Plotly.js script
app = Dash(
    __name__,
    external_scripts=[
        "https://cdn.plot.ly/plotly-2.35.0.min.js"  # Explicitly load Plotly.js from CDN
    ]
)

# Load the Excel data
df = pd.read_excel("rrg_sectoral.xlsx", parse_dates=["DATES"], index_col="DATES")

# Define sectors and benchmark options
sectors = ["Auto", "Bank", "Chemicals", "Consumer Durables", "FMCG", "Healthcare", 
           "Pharma", "IT", "Metal", "NSELV30 Index", "N200MOMP Index", 
           "NSEQ30TR Index", "NSENV20 Index"]
default_benchmark = "NIFTY Index"

# App layout
app.layout = html.Div([
    html.H1("RRG Sector Analysis", className="text-2xl font-bold mb-4"),
    html.Div([
        html.Label("Select Sectors", className="block text-sm font-medium text-gray-700"),
        dcc.Dropdown(
            id="sector-select",
            options=[{"label": sector, "value": sector} for sector in sectors],
            value=["FMCG"],
            multi=True,
            className="mb-4"
        ),
    ]),
    html.Div([
        html.Label("Select Benchmark", className="block text-sm font-medium text-gray-700"),
        dcc.Dropdown(
            id="benchmark-select",
            options=[{"label": bm, "value": bm} for bm in [default_benchmark]],
            value=default_benchmark,
            className="mb-4"
        ),
    ]),
    html.Div([
        html.Label(id="rolling-window-label", className="block text-sm font-medium text-gray-700"),
        dcc.Slider(
            id="rolling-window",
            min=10,
            max=100,
            step=1,
            value=64,
            marks={i: str(i) for i in range(10, 101, 10)},
            className="mb-4"
        ),
    ]),
    html.Div([
        html.Label(id="slice-window-label", className="block text-sm font-medium text-gray-700"),
        dcc.Slider(
            id="slice-window",
            min=1,
            max=20,
            step=1,
            value=5,
            marks={i: str(i) for i in range(1, 21, 2)},
            className="mb-4"
        ),
    ]),
    html.Div([
        html.Label(id="tail-length-label", className="block text-sm font-medium text-gray-700"),
        dcc.Slider(
            id="tail-length",
            min=5,
            max=20,
            step=1,
            value=10,
            marks={i: str(i) for i in range(5, 21, 2)},
            className="mb-4"
        ),
    ]),
    # OPTION 1: Add animation controls
    html.Div([
        html.Button("Play/Pause Animation", id="play-pause-btn", className="mr-2 px-4 py-2 bg-blue-500 text-white rounded"),
        html.Button("Show Latest", id="show-latest-btn", className="px-4 py-2 bg-green-500 text-white rounded"),
    ], className="mb-4"),
    dcc.Loading(
        id="loading",
        type="circle",
        children=[
            dcc.Graph(id="rrg-plot")
        ]
    ),
    # OPTION 1: Make interval conditional
    dcc.Interval(id="interval-component", interval=2000, n_intervals=0, disabled=True),  # Start disabled, slower interval
    # OPTION 2: Add a store to track animation state
    dcc.Store(id="animation-state", data={"playing": False, "show_latest": True})
], className="container mx-auto p-4")

# Callback to update slider labels
@app.callback(
    [
        Output("rolling-window-label", "children"),
        Output("slice-window-label", "children"),
        Output("tail-length-label", "children")
    ],
    [
        Input("rolling-window", "value"),
        Input("slice-window", "value"),
        Input("tail-length", "value")
    ]
)
def update_slider_labels(rolling_window, slice_window, tail_length):
    return (
        f"Rolling Window: {rolling_window}",
        f"Slice Window: {slice_window}",
        f"Tail Length: {tail_length}"
    )

# Callback to handle animation controls
@app.callback(
    [
        Output("animation-state", "data"),
        Output("interval-component", "disabled")
    ],
    [
        Input("play-pause-btn", "n_clicks"),
        Input("show-latest-btn", "n_clicks")
    ],
    prevent_initial_call=True
)
def control_animation(play_pause_clicks, show_latest_clicks):
    from dash.callback_context import triggered_id
    
    if triggered_id == "play-pause-btn":
        # Toggle animation
        return {"playing": True, "show_latest": False}, False
    elif triggered_id == "show-latest-btn":
        # Show latest data only
        return {"playing": False, "show_latest": True}, True
    
    return {"playing": False, "show_latest": True}, True

# Callback to update the RRG plot
@app.callback(
    Output("rrg-plot", "figure"),
    [
        Input("sector-select", "value"),
        Input("benchmark-select", "value"),
        Input("rolling-window", "value"),
        Input("slice-window", "value"),
        Input("tail-length", "value"),
        Input("interval-component", "n_intervals"),
        Input("animation-state", "data")
    ]
)
def update_rrg_plot(selected_sectors, benchmark, rolling_window, slice_window, tail_length, n_intervals, animation_state):
    if not selected_sectors or benchmark not in df.columns:
        return go.Figure()  # Return empty figure if inputs are invalid

    # Calculate Relative Strength Ratio (price of sector / price of benchmark)
    rs_ratios = df[selected_sectors].div(df[benchmark], axis=0)
    short_period = 5
    rs_momentum = (rs_ratios / rs_ratios.shift(short_period) - 1) * 100

    # Normalize RS Ratios
    rs_ratios_normalized = rs_ratios.rolling(window=rolling_window).apply(
        lambda x: (x.iloc[-1] - x.mean()) / x.std() if len(x.dropna()) > 1 else np.nan
    )
    rs_ratios_normalized = rs_ratios_normalized.iloc[::slice_window]

    # Normalize RS Momentum
    rs_momentum_normalized = rs_momentum.rolling(window=rolling_window).apply(
        lambda x: (x.iloc[-1] - x.mean()) / x.std() if len(x.dropna()) > 1 else np.nan
    )
    rs_momentum_normalized = rs_momentum_normalized.iloc[::slice_window]

    # Filter out rows with all NaN values
    valid_data = rs_ratios_normalized.dropna(how='all')
    valid_momentum = rs_momentum_normalized.dropna(how='all')
    
    if len(valid_data) == 0:
        return go.Figure().add_annotation(
            text="No valid data available",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )

    # Determine which date to show
    if animation_state.get("show_latest", True):
        # Show latest available data
        target_date = valid_data.index[-1]
    else:
        # Animation mode: cycle through valid dates only
        valid_dates = valid_data.index
        current_frame = n_intervals % len(valid_dates)
        target_date = valid_dates[current_frame]

    # Get data for the target date
    rs_ratios_latest = rs_ratios_normalized.loc[target_date]
    rs_momentum_latest = rs_momentum_normalized.loc[target_date]

    # Filter out sectors with NaN values for this date
    valid_sectors = [s for s in selected_sectors if not pd.isna(rs_ratios_latest[s]) and not pd.isna(rs_momentum_latest[s])]
    
    if not valid_sectors:
        return go.Figure().add_annotation(
            text=f"No valid data for {target_date.strftime('%Y-%m-%d')}",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )

    # Create the RRG plot
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly[:len(valid_sectors)]

    # Plot scatter points and tails for each valid sector
    for sector, color in zip(valid_sectors, colors):
        # Tail: Plot the last tail_length points (only valid ones)
        sector_data = rs_ratios_normalized[sector].dropna()
        sector_momentum = rs_momentum_normalized[sector].dropna()
        
        # Align the data
        common_dates = sector_data.index.intersection(sector_momentum.index)
        if len(common_dates) < 2:
            continue
            
        tail_data_rs = sector_data.loc[common_dates].iloc[-tail_length:]
        tail_data_momentum = sector_momentum.loc[common_dates].iloc[-tail_length:]

        if len(tail_data_rs) > 1:
            # Plot the tail line with spline interpolation
            fig.add_trace(
                go.Scatter(
                    x=tail_data_rs,
                    y=tail_data_momentum,
                    mode="lines",
                    line=dict(color=color, width=3, shape="spline"),
                    showlegend=False,
                    name=f"{sector}_tail"
                )
            )

            # Add arrows along the path (simplified)
            if len(tail_data_rs) > 3:
                x1, y1 = tail_data_rs.iloc[-2], tail_data_momentum.iloc[-2]
                x2, y2 = tail_data_rs.iloc[-1], tail_data_momentum.iloc[-1]
                dx = x2 - x1
                dy = y2 - y1
                if abs(dx) > 0.01 or abs(dy) > 0.01:  # Only add arrow if there's meaningful movement
                    length = np.sqrt(dx**2 + dy**2)
                    if length > 0:
                        arrow_length = min(0.15, length * 0.3)
                        dx_norm = (dx / length) * arrow_length
                        dy_norm = (dy / length) * arrow_length
                        fig.add_annotation(
                            x=x2,
                            y=y2,
                            ax=x2 - dx_norm,
                            ay=y2 - dy_norm,
                            xref="x",
                            yref="y",
                            axref="x",
                            ayref="y",
                            showarrow=True,
                            arrowhead=3,
                            arrowsize=2,
                            arrowwidth=3,
                            arrowcolor=color
                        )

        # Scatter point for the current frame
        fig.add_trace(
            go.Scatter(
                x=[rs_ratios_latest[sector]],
                y=[rs_momentum_latest[sector]],
                mode="markers+text",
                name=sector,
                marker=dict(size=15, color=color, symbol="circle", 
                          line=dict(width=2, color="white")),
                text=[sector],
                textposition="top center",
                textfont=dict(size=12, color="black")
            )
        )

    # Add quadrant lines and backgrounds
    fig.add_shape(type="line", x0=0, y0=-3, x1=0, y1=3, line=dict(color="black", width=1))
    fig.add_shape(type="line", x0=-3, y0=0, x1=3, y1=0, line=dict(color="black", width=1))
    fig.add_shape(type="rect", x0=0, y0=0, x1=3, y1=3, fillcolor="lightgreen", opacity=0.3, layer="below")
    fig.add_shape(type="rect", x0=0, y0=0, x1=3, y1=-3, fillcolor="yellow", opacity=0.3, layer="below")
    fig.add_shape(type="rect", x0=0, y0=0, x1=-3, y1=-3, fillcolor="lightcoral", opacity=0.3, layer="below")
    fig.add_shape(type="rect", x0=0, y0=0, x1=-3, y1=3, fillcolor="lightblue", opacity=0.3, layer="below")

    # Add quadrant labels
    fig.add_annotation(x=1.5, y=1.5, text="Leading<br>(Strong & Improving)", 
                      showarrow=False, font=dict(size=10, color="darkgreen"))
    fig.add_annotation(x=1.5, y=-1.5, text="Weakening<br>(Strong & Deteriorating)", 
                      showarrow=False, font=dict(size=10, color="orange"))
    fig.add_annotation(x=-1.5, y=-1.5, text="Lagging<br>(Weak & Deteriorating)", 
                      showarrow=False, font=dict(size=10, color="darkred"))
    fig.add_annotation(x=-1.5, y=1.5, text="Improving<br>(Weak & Improving)", 
                      showarrow=False, font=dict(size=10, color="blue"))

    # Update layout
    animation_status = "Playing" if animation_state.get("playing", False) else "Static"
    fig.update_layout(
        title=f"Relative Rotation Graph vs. {benchmark} ({target_date.strftime('%Y-%m-%d')}) - {animation_status}",
        xaxis_title="Relative Strength Ratio (Z-score)",
        yaxis_title="Relative Strength Momentum (Z-score)",
        showlegend=True,
        width=900,
        height=700,
        xaxis=dict(range=[-3, 3], zeroline=True),
        yaxis=dict(range=[-3, 3], zeroline=True),
        margin=dict(t=100)
    )

    return fig

# Run the app
if __name__ == "__main__":
    app.run(debug=True)

[2025-06-06 21:11:08,845] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Harsha.1Aditya\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Harsha.1Aditya\AppData\Local\Programs\Python\Python313\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "c:\Users\Harsha.1Aditya\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py", line 1414, in dispatch
    ctx.run(
    ~~~~~~~^
        functools.partial(
        ^^^^^^^^^^^^^^^^^^
    ...<7 lines>...
        )
        ^
    )
    ^
  File "c:\Users\Harsha.1Aditya\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\_callback.py", line 536, in add_c

In [176]:
pip install dash_bootstrap_components

Note: you may need to restart the kernel to use updated packages.
